In [11]:
import torch

In [12]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [13]:
x.requires_grad_(True) # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad # 默认值是None

In [14]:
# 计算x和x的点积，赋值给y的标量输出
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [15]:
# 通过调⽤反向传播函数来⾃动计算y关于x每个分量的梯度
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [16]:
# 验证梯度是否计算正确
x.grad == 4 * x

tensor([True, True, True, True])

In [17]:
# 计算x的另⼀个函数
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

⾮标量变量的反向传播
* 当y不是标量时，向量y关于向量x的导数的最⾃然解释是⼀个矩阵。
* 对于⾼阶和⾼维的y和x，求导的结果可以是⼀个⾼阶张量。
* ⽬的不是计算微分矩阵，⽽是单独计算批量中每个样本的偏导数之和。

In [18]:
# 对⾮标量调⽤backward需要传⼊⼀个gradient参数，该参数指定微分函数关于self的梯度。
# 在我们的例⼦中，我们只想求偏导数的和，所以传递⼀个1的梯度是合适的
x.grad.zero_()
y = x * x

In [19]:
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

分离计算

In [20]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [21]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

Python控制流的梯度计算

In [25]:
def f(a): # while循环的迭代次数和if语句的结果都取决于输⼊a的值
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c


In [23]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

* 它在其输⼊a中是分段线性的。换⾔之，对于任何a，存在某个常量标量k，使得f(a)=k*a，其中k的值取决于输⼊a。
* 因此，我们可以⽤d/a验证梯度是否正确。

In [24]:
a.grad == d / a

tensor(True)